In [155]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import csv
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.naive_bayes import GaussianNB,BernoulliNB, ComplementNB, MultinomialNB
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier


# EDA

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train_X = train.drop(labels = 'target', axis = 1)
train_y = train.drop(labels = ['id','keyword','location','text'], axis = 1)

In [4]:
print("Dimention of train data : ",train_X.shape)
print("Dimention of train label : ",train_y.shape)
print("Dimention of test data : ",test.shape)

Dimention of train data :  (7613, 4)
Dimention of train label :  (7613, 1)
Dimention of test data :  (3263, 4)


In [5]:
#Null Data Field
for c in train_X.columns:   
    print("Number of Null Cells in ---",c,"----are ",train_X[c].isnull().sum())

Number of Null Cells in --- id ----are  0
Number of Null Cells in --- keyword ----are  61
Number of Null Cells in --- location ----are  2533
Number of Null Cells in --- text ----are  0


In [6]:
Total_data = pd.concat([train_X, test], ignore_index=True)

In [7]:
Total_data

,id,keyword,location,text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...
...,...,...,...,...
10871,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
10872,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
10873,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
10874,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


NameError: name 'text' is not defined

# PreProcessing

In [73]:
def tags(data):
    pattern = re.compile(r'[@|#][a-zA-Z]+')
    matches = pattern.findall(data)
    tags = [match[1:].lower() for match in matches]
    return ' '.join(tags)

def striplinks(data):
    cleanr = re.compile(r'https?://\S+|www\.\S+')
    clean = re.sub(cleanr, ' ', data)
    return clean
    
def alphanumeric(data):
    cleanr = re.compile(r'[^A-Za-z]+')
    clean = re.sub(cleanr, ' ', data)
    return clean


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

In [10]:
stop_words = set(stopwords.words('english'))
stemmer  = SnowballStemmer('english')

In [74]:
start = datetime.now()
with open('processed_total_data.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['tweets'])
    for i in tqdm(range(10876)):
        
        #raw data
        keyword = str(Total_data['keyword'][i])
        location = str(Total_data['location'][i])
        text = str(Total_data['text'][i])
   
        #extract tags 
        tags_str = stemmer.stem(tags(text))
        #print(tags_str)
        #strip links and alphanumberic charcters
        text_sl = striplinks(text)
        text_sl_an = alphanumeric(text_sl)
        #print(text_sl)
        #print(text_sl_an)
        #removing stopwords and stem the text       
        text_words = word_tokenize(text_sl_an.lower())
        text_sl_an_sw_st = ' '.join(stemmer.stem(j) for j in text_words if j not in stop_words )
        #print(text_words)
        #print(text_sl_an_sw_st)
        #assigning weights
        processed_list = []
        if(tags_str != ''):
            processed_list.extend([tags_str,tags_str,tags_str,text_sl_an_sw_st])
        else:
            processed_list.extend([text_sl_an_sw_st])
        if (keyword != 'nan'):
            #print(keyword)
            processed_list.extend([keyword,keyword])
        if (location != 'nan'):
            location_words = word_tokenize(alphanumeric(location))
            location_str = " ".join(location_words)
            processed_list.extend([location_str])
        
        #saving string
        
        processed_string = " ".join(processed_list)

        writer.writerow([processed_string])
print("Time to run this subsection :", datetime.now() - start)

100%|██████████████████████████████████████████████████████████████████████████| 10876/10876 [00:05<00:00, 1871.92it/s]


Time to run this subsection : 0:00:05.826058


In [ ]:
Total_data['keyword'][0] == float('nan')

In [ ]:
str(Total_data['keyword'][0]) == 'nans'

'painting is funny thing swim'

In [26]:
print(stop_words)

{"weren't", 'such', "she's", 'these', 'if', 'those', 'and', 'here', 'should', 'its', 'itself', 'i', 'll', 'ours', 'then', 'ma', 'further', 'to', "haven't", 't', 'each', 'hadn', 'my', 'while', 'she', 'into', 'him', "you're", 'me', 'so', "hadn't", "isn't", 'd', 'out', 'myself', 'do', 'himself', "aren't", 'ourselves', 'when', 'why', 'in', 'too', 'doing', 'until', "you'd", "shouldn't", 'isn', 'than', 're', 'her', 'not', "couldn't", 'are', "it's", 'be', 'it', 'just', "needn't", 'yourselves', 'had', 'below', 'their', 'up', 'your', "that'll", 'aren', 'was', 'm', 'only', 'or', 'that', 'did', 'a', "wasn't", "you've", 'by', 'between', 'more', 'we', 'being', 'same', 'herself', 'am', 'down', 'don', 'y', 'wouldn', 'no', 's', 'against', 'very', 'any', 'theirs', 'how', 'ain', 'through', 'hasn', "mightn't", 'haven', 'all', 'which', 'other', "you'll", 'at', 'them', "hasn't", 'an', 'does', 'now', 'his', 'some', 'themselves', 'whom', 'once', 'shouldn', 'will', 'most', 'couldn', 'been', 'the', 'but', 'own

# Machine learning

## Featurizing data with TF-IDF

In [75]:
processed_total_data = pd.read_csv("processed_total_data.csv")
processed_total_data.replace(np.nan, '', regex=True, inplace = True)

In [81]:
start = datetime.now()
#smooth_idf=True by default so smoothing is done by defult.
#norm is l2 by default.
#subliner is used False by default.
vectorizer = TfidfVectorizer(min_df = 0.0001, 
                             max_features = 100000, 
                             tokenizer = lambda x: x.split(),
                             ngram_range = (1,4))
total_tfidf = vectorizer.fit_transform( processed_total_data['tweets'])

print("Time to run this Subsection ", datetime.now() - start)

Time to run this Subsection  0:00:01.399283


In [82]:
train_tfidf.shape

(10876, 35879)

## Sampling Training Data

In [87]:
train_tfidf = total_tfidf[0:7613]
X_train, X_test, y_train, y_test = train_test_split(train_tfidf, train_y, 
                                                    test_size = 0.2, random_state = 42)

## Supervised Learning Algorithms

### Logistic Regression

Model 1: C=1;penalty='l2'

In [128]:
Model1 = LogisticRegression(C=1,penalty='l2')
Model1.fit(X_train, y_train)
y_pred = Model1.predict(X_test)

print("Acuracy", accuracy_score(y_test['target'], y_pred))
print("Macro precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='macro'))
print("Micro precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='micro'))
print("Weighted precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='weighted'))

Acuracy 0.7918581746552856
Macro precision_recall_fscore_support
(0.7907704684243244, 0.7803714921389358, 0.7838016598502615, None)
Micro precision_recall_fscore_support
(0.7918581746552856, 0.7918581746552856, 0.7918581746552856, None)
Weighted precision_recall_fscore_support
(0.7914446665013665, 0.7918581746552856, 0.7899673599561473, None)


### Naive Bayes 

Model 2.1: Gaussian Naive Bayes
Model 2.2: Bernoulli Naive Bayes
Model 2.3: Complement Naive Bayes
Model 2.4: Multinomial Naive Bayes

In [131]:
Model21 = GaussianNB()
Model21.fit(X_train.toarray(), y_train)
y_pred = Model21.predict(X_test.toarray())

print("Acuracy", accuracy_score(y_test['target'], y_pred))
print("Macro precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='macro'))
print("Micro precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='micro'))
print("Weighted precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='weighted'))

Acuracy 0.7255416940249507
Macro precision_recall_fscore_support
(0.7203331995182658, 0.712872823178063, 0.7151421483698219, None)
Micro precision_recall_fscore_support
(0.7255416940249507, 0.7255416940249507, 0.7255416940249507, None)
Weighted precision_recall_fscore_support
(0.7236158641233194, 0.7255416940249507, 0.7231830341856432, None)


In [134]:
Model22 = BernoulliNB()
Model22.fit(X_train.toarray(), y_train)
y_pred = Model22.predict(X_test.toarray())

print("Acuracy", accuracy_score(y_test['target'], y_pred))
print("Macro precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='macro'))
print("Micro precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='micro'))
print("Weighted precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='weighted'))

Acuracy 0.7734734077478661
Macro precision_recall_fscore_support
(0.8148599786170851, 0.741941483641441, 0.7470787714835148, None)
Micro precision_recall_fscore_support
(0.7734734077478661, 0.7734734077478661, 0.7734734077478661, None)
Weighted precision_recall_fscore_support
(0.8025912323119806, 0.7734734077478661, 0.759149489287334, None)


In [136]:
Model23 = ComplementNB()
Model23.fit(X_train.toarray(), y_train)
y_pred = Model23.predict(X_test.toarray())

print("Acuracy", accuracy_score(y_test['target'], y_pred))
print("Macro precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='macro'))
print("Micro precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='micro'))
print("Weighted precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='weighted'))

Acuracy 0.788575180564675
Macro precision_recall_fscore_support
(0.7873781505225257, 0.7769160793052505, 0.780333652257734, None)
Micro precision_recall_fscore_support
(0.788575180564675, 0.788575180564675, 0.788575180564675, None)
Weighted precision_recall_fscore_support
(0.7881160457539876, 0.788575180564675, 0.7866195636782821, None)


In [139]:
Model24= MultinomialNB()
Model24.fit(X_train.toarray(), y_train)
y_pred = Model24.predict(X_test.toarray())

print("Acuracy", accuracy_score(y_test['target'], y_pred))
print("Macro precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='macro'))
print("Micro precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='micro'))
print("Weighted precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='weighted'))

Acuracy 0.7898883782009193
Macro precision_recall_fscore_support
(0.8004832111577849, 0.7701268982733513, 0.7761734178377881, None)
Micro precision_recall_fscore_support
(0.7898883782009193, 0.7898883782009193, 0.7898883782009194, None)
Weighted precision_recall_fscore_support
(0.7959769022441842, 0.7898883782009193, 0.7843587390094182, None)


### Support Vector Machines

In [143]:
Model31 = svm.SVC(kernel='rbf') #rbf by default svm.SVC()
Model31.fit(X_train, y_train)
y_pred = Model31.predict(X_test)

print("Acuracy", accuracy_score(y_test['target'], y_pred))
print("Macro precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='macro'))
print("Micro precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='micro'))
print("Weighted precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='weighted'))

Acuracy 0.5738673670387393
Macro precision_recall_fscore_support
(0.28693368351936965, 0.5, 0.3646224447225699, None)
Micro precision_recall_fscore_support
(0.5738673670387393, 0.5738673670387393, 0.5738673670387393, None)
Weighted precision_recall_fscore_support
(0.32932375495197513, 0.5738673670387393, 0.4184898446323389, None)


In [144]:
Model32 = svm.SVC(kernel='linear')
Model32.fit(X_train, y_train)
y_pred = Model32.predict(X_test)

print("Acuracy", accuracy_score(y_test['target'], y_pred))
print("Macro precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='macro'))
print("Micro precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='micro'))
print("Weighted precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='weighted'))

Acuracy 0.7826657912015759
Macro precision_recall_fscore_support
(0.7802441549948226, 0.7719656715312768, 0.7748362680688006, None)
Micro precision_recall_fscore_support
(0.7826657912015759, 0.7826657912015759, 0.7826657912015759, None)
Weighted precision_recall_fscore_support
(0.7818326918651418, 0.7826657912015759, 0.7810392353394852, None)


### RandomForestClassifier

In [152]:
Model41 = RandomForestClassifier(random_state=0)
Model41.fit(X_train, y_train)
y_pred = Model41.predict(X_test)

print("Acuracy", accuracy_score(y_test['target'], y_pred))
print("Macro precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='macro'))
print("Micro precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='micro'))
print("Weighted precision_recall_fscore_support")
print(precision_recall_fscore_support(y_test['target'], y_pred, average='weighted'))

Acuracy 0.7518056467498359
Macro precision_recall_fscore_support
(0.7613299103996778, 0.7288144408048995, 0.7333179539061891, None)
Micro precision_recall_fscore_support
(0.7518056467498359, 0.7518056467498359, 0.7518056467498359, None)
Weighted precision_recall_fscore_support
(0.7576159427719232, 0.7518056467498359, 0.74369134764639, None)
